In [ ]:
INF = float('inf')
import time, random, string
from collections import defaultdict

In [ ]:
class GeneticAlgorithm:
  def __init__(self, pop, ideal, n, bits, cross_rate, mutation_rate):
    self.pop = pop
    self.n_pop = len(pop)
    self.ideal = ideal
    self.n = n
    self.bits = bits
    self.cross_rate = cross_rate
    self.mutation_rate = mutation_rate

  def fitness(self, chromosome):
    return sum( [int(c) * 2**(self.bits - i - 1) for i, c in enumerate(chromosome)] )

  def select(self, k=3):
    selection = random.choice(self.pop)
    for p in random.choices(self.pop, k=k):
      if self.fitness(p) > self.fitness(selection):
        selection = p
    
    return selection

  def crossover(self,p1, p2):
    pt = len(p1)
    if random.random() < self.cross_rate:
      pt = random.randint(1, len(p1)-2)

    return [
      p1[:pt] + p2[pt:],
      p2[:pt] + p1[pt:]
    ]

  def mutation(self, c):
    if random.random() < self.mutation_rate:
      idx = random.randint(0, len(c)-1)
      c[idx] = str(1 - int(c[idx]))
    return c
  
  def run(self, debug=False):
    start_time = time.time_ns()
    best = random.choice(self.pop)

    for i in range(self.n):
      oldbest = best
      best = max(best, *self.pop, key = lambda chromosome: self.fitness(chromosome))
      if debug and best > oldbest:
        print(f"[{i}] {''.join(best), self.fitness(best)}")
      if best == self.ideal:
        break

      selected_pairs = [(self.select(), self.select()) for _ in range(int(self.n_pop/2))]
      children = []
      for p1, p2 in selected_pairs:
        c1, c2 = self.crossover(p1, p2)
        c1, c2 = self.mutation(c1), self.mutation(c2)
        children.extend((c1, c2))
      
      self.pop = children

    return i, ''.join(best), (time.time_ns() - start_time)/1e6, best == self.ideal

## Task 1

In [ ]:
class TargetString(GeneticAlgorithm):
  def __init__(self, target):
    self.pop_range = string.ascii_letters + string.punctuation + string.digits + ' '
    n_pop = 70
    pop = [random.choices(self.pop_range, k=len(target)) for _ in range(n_pop)]
    super().__init__(pop, list(target), 100000, len(target), 0.9, 3.0/float(len(target)))

  def fitness(self, chromosome):
    return len( [True for i, j in zip(chromosome, self.ideal) if i == j] )
  
  def mutation(self,c):
    if random.random() < self.mutation_rate:
      idx = random.randint(0, len(c)-1)
      c[idx] = random.choice(self.pop_range.replace(c[idx], ''))
    return c

t = TargetString("Artificial Intelligence Lab")

print("Best chromosomes Progression")
gens, result, time_taken, found = t.run(True)
print(f"Time taken: {time_taken}ms")


Best chromosomes Progression
[3] ('A3;Lf}4-5lNvXaYq&Lg=>y/43m{', 4)
[7] ("Ai7Lf}4-5lNvK,e6lsQ'H:U,L,w", 6)
[12] ("A3tLf}4-5lNvf,e6lLg'>1/ L,w", 9)
[21] ("A3tLf94-5l vXae6lLg'Hf/ Law", 11)
[60] ("A3tLf}4:5l vXae6lLg'gf/ Lab", 12)
[83] ('Aatrf94-5l vXae6lLge7f/ Lab', 13)
[125] ('A3trfi[-al vXaellLge7c/ Lab', 17)
[194] ('A3tific-al vXaellLge7c/ Lab', 19)
[277] ('Artific-al LXaellLgenc/ Lab', 21)
[449] ('Artific<al LXtellLgencE Lab', 22)
[461] ('Artificial LXtellLgencE Lab', 23)
[639] ('Artificial Lntell(gence Lab', 25)
[644] ('Artificial Lntelligence Lab', 26)
Time taken: 3271.065762ms


## Task 3

In [ ]:
class Node:
  def __init__(self, key, cost, h):
    self.data = key
    self.cost = cost
    self.h = h
    self.left = None
    self.right = None

root = Node('S', 0, 13)
root.left = Node('A', 3, 12)
root.right = Node('B', 2, 4)
root.left.left = Node('C', 4, 7)
root.left.right = Node('D', 1, 3)
root.right.left = Node('E', 3, 8)
root.right.right = Node('F', 1, 2)
root.right.left.left = Node('H', 5, 4)
root.right.right.left = Node('I', 2, 9)
root.right.right.right = Node('G', 3, 0)


In [ ]:
def greedy_best_first(root, src, target):
  path = [root.data]
  total_cost = 0
  bound = 100
	
  while bound:
    root = min(root.left, root.right, key=lambda node: node.h)
    cost = root.cost
    total_cost += cost
    path.append(root.data)
    if root.data == target:
      return path, total_cost
    bound -= 1

path, cost = greedy_best_first(root, 'S', 'G')
print("Path:", ' -> '.join(path))
print("Cost:", cost)


Path: S -> B -> F -> G
Cost: 6


## Task 4

In [ ]:
class Edge:
  def __init__(self, src, target, g):
    self.src = src
    self.target = target
    self.g = g

# This class represents a directed graph using adjacency list representation
class Graph:
  def __init__(self, vertices, heuristic):
    self.heuristic = heuristic
    self.V = vertices
    self.graph = defaultdict(lambda: [])

  def addEdge(self, u, v, cost):
    self.graph[u].append(Edge(u, v, cost))
  
  def a_star(self, src, target):
    open = [ Edge(None, src, 0) ]
    min_cost = defaultdict(lambda: INF)
    min_cost[src] = 0
    closed = [src]

    while open:
      open.sort(key = lambda x: min_cost[x.target] + self.h(x.target))
      edge = open.pop(0)
      closed.append(edge.target)

      if edge.target == target:
        return min_cost[target]
      
      for child in self.graph[edge.target]:
        if child in closed:
          continue
        if child not in open:
          open.append(child)
        min_cost[child.target] = min(min_cost[child.target], min_cost[edge.target] + child.g)


  def h(self, x):
    return self.heuristic[x]
  

## Task 4

In [ ]:
edges = [
         ('S', 'A', 1), ('S', 'G', 10), ('A', 'C', 1), ('A', 'B', 2),
         ('B', 'D', 5), ('C', 'D', 3), ('C', 'G', 4), ('D', 'G', 3) # Assumed. No cost included in graph for D->G
]

h = {
    'S': 5,
    'A': 3,
    'B': 4,
    'C': 2,
    'D': 6,
    'G': 0
}

graph = Graph(6, h)
for u, v, cost in edges:
  graph.addEdge(u, v, cost)


print(f"Minimum cost: {graph.a_star('S', 'G')}")

Minimum cost: 6


## Task 5

In [ ]:
class Node:
    def __init__(self, position, h, parent=None):
        self.position = position
        self.parent = parent
        self.g = 1
        self.h = h
        self.f = INF
    
    def __eq__(self, other):
        return self.position == other.position
    
    def __lt__(self, other):
        return self.f + self.h < other.f + self.h
    
    def __repr__(self):
        return (f"{self.position}  {self.f}  {self.h}")


maze = [
     [INF, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
     [INF, 11, INF, INF, INF, INF, INF, INF, INF, INF, INF, 1],
     [INF, 12, INF, 10, 9, 8, 7, 6, 5, 4, INF, 2],
     [INF, 13, INF, 11, INF, INF, INF, INF, INF, 5, INF, 3],
     [INF, 14, 13, 12, INF, 10, 9, 8, 7, 6, INF, 4],
     [INF, INF, INF, 13, INF, 11, INF, INF, INF, INF, INF, 5],
     [17, 16, 15, 14, INF, 12, 11, 10, 9, 8, 7, 6]
]

maze = { (i, j) : Node((i,j), key) for i in range(len(maze)) for j, key in enumerate(maze[i])}

In [ ]:


def a_star(start, goal):
  start.f = 0
  open = [ start ]
  closed = []

  while open:
    open.sort()
    node = open.pop(0)
    closed.append(node)
    
    if node == goal:
      path = []
      while node:
        path.append(node.position)
        node = node.parent
      return list(reversed(path)), goal.f
    
    x, y = node.position
    neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]

    for pos in neighbors:
      neigh = maze.get(pos)
      if not neigh or neigh.h == INF or neigh in closed: continue
      # print(neigh.position, end=' ')
      if neigh.f > node.f + neigh.g:
        neigh.f = node.f + neigh.g
        neigh.parent = node
      # print(node.f, neigh.g)
      if neigh not in open:
        open.append(neigh)

path, cost = a_star(maze[(6, 0)], maze[(0, 11)])
print(f"Path: {' -> '.join([str(t) for t in path])}")
print(f"Cost: {cost}")

Path: (6, 0) -> (6, 1) -> (6, 2) -> (6, 3) -> (5, 3) -> (4, 3) -> (4, 2) -> (4, 1) -> (3, 1) -> (2, 1) -> (1, 1) -> (0, 1) -> (0, 2) -> (0, 3) -> (0, 4) -> (0, 5) -> (0, 6) -> (0, 7) -> (0, 8) -> (0, 9) -> (0, 10) -> (0, 11)
Cost: 21
